## Project 1: Resume Parser

### A simple example of a commercial use case: parsing and summarizing a resume

This directory includes a sample JSON file, along with a text version of my resume.

We will be using a 'one shot prompt' style with OpenAI, and using Gradio to make a quick UI.

(Reminder: I'll be stepping through this code at a high level to give you a general sense, so you can try this yourself afterwards.)

In [ ]:
# imports

import os
from dotenv import load_dotenv
import openai
import json
from openai import OpenAI

In [ ]:
# constants

MODEL = "gpt-4o"

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# Load sample HR-JSON file

with open('sample.json', 'r') as f:
    sample = f.read()

In [ ]:
# Also load my resume as an example

with open('my_resume.txt', 'r') as f:
    me = f.read()

In [ ]:
# Create the OpenAI client

client = OpenAI()

In [ ]:
# Create the prompt, comprising of system instructions, and a specific request

instructions = f"""You are an assistant that can convert a resume in plain text precisely into the industry standard HR-JSON format.
Here is an example of HR-JSON populated for a candidate.
HR-JSON elements are optional, so if a section isn't populated in a resume, then it should be omitted in the HR-JSON.
Example HR-JSON:\n{sample}"""

request = "Please convert this resume to HR-JSON:\n"

In [ ]:
# Let's try it for my resume
# Note the use of response_format to ensure JSON response
# And the use of the stream=True to have the results flow back

response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": request + me}
          ],
        response_format={ "type": "json_object" },
        stream=True)

for chunk in response:
    delta = chunk.choices[0].delta.content or ''
    print(delta, end='')

In [ ]:
# Wrap this in a function (a generator)

def parse(text_resume):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": request + text_resume}
          ],
        response_format={ "type": "json_object" },
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# Bringing up a view in Gradio is incredibly easy!

import gradio as gr

view = gr.Interface(
    fn=parse,
    inputs=[gr.Textbox(label="Text Resume", lines=10)],
    outputs=[gr.Textbox(label="HR-JSON Resume", lines=10)],
    allow_flagging="never"
)

view.launch()

In [ ]:
# A simple variation to generate a summary of this resume, a very common Gen AI use case

summary_instructions = """You are an assistant that can create a short summary of a candidate based on their resume.
The summary should be 4-5 sentences long and capture their experience and education."""

summary_request = "Please summarize this candidate based on their resume:\n"

In [ ]:
# Generate as before

def summarize(text_resume):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": summary_instructions},
            {"role": "user", "content": summary_request + text_resume}
          ],
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# And in Gradio again

view = gr.Interface(
    fn=summarize,
    inputs=[gr.Textbox(label="Text Resume", lines=10)],
    outputs=[gr.Textbox(label="Summary", lines=10)],
    allow_flagging="never"
)

view.launch()

In [ ]:
# One more example: write a cover letter

instructions = """You are an assistant that can write a short cover letter that a candidate can use when applying for a job.
They will send the cover letter to the hiring manager, enclosing their resume"""

In [ ]:
# Generate as before

def cover(text_resume, text_job):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": f'Please write a cover letter for the following candidate, applying to the following job.\nCandidate resume:\n\n{text_resume}\n\nJob description:\n\n{text_job}'}
          ],
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# And in Gradio again

view = gr.Interface(
    fn=cover,
    inputs=[gr.Textbox(label="Text Resume", lines=5), gr.Textbox(label="Job Description", lines=5)],
    outputs=[gr.Textbox(label="Cover letter", lines=20)],
    allow_flagging="never"
)

view.launch()

## Splitting into multiple LLM calls - a light version of Agents

I found the cover letter was quite verbose. I experimented with a different approach. Let's break this into 2 LLM calls; first summarize the candidate, then use the summary to generate the cover letter.

In [ ]:
def summary_agent(text_resume):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": summary_instructions},
            {"role": "user", "content": summary_request + text_resume}
          ])
    return response.choices[0].message.content

In [ ]:
def cover_with_summary_agent(text_resume, text_job):
    summary = summary_agent(text_resume)
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": f'Please write a short cover letter for the following candidate, applying to the following job. Keep the cover letter to 2 paragraphs maximum.\nCandidate summary:\n\n{summary}\n\nJob description:\n\n{text_job}'}
          ],
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# And in Gradio again

view = gr.Interface(
    fn=cover_with_summary_agent,
    inputs=[gr.Textbox(label="Text Resume", lines=5), gr.Textbox(label="Job Description", lines=5)],
    outputs=[gr.Textbox(label="Cover letter", lines=12)],
    allow_flagging="never"
)

view.launch()

# Further exercises

1. In the original Resume Parser, try removing the one-shot example in the prompt. What happens?
2. Try parsing your own resume!
3. To be more robust, we would need to provide multiple examples in the prompt including a candidate and its JSON. Give this a try.
4. OpenAI have recently announced the ability to specify a JSON schema to be used for the response. Find out about this and add it to the example! (And if you'd like to show me, please submit a PR so I can add it for the future!)